In [ ]:
%matplotlib inline

from database.DataBaseModels import *
from source.ArticleParser import ArticleParser
from itertools import combinations
from collections import Counter

import pygal
import operator
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
total_digraph = nx.MultiDiGraph()


for citation in db.Citation.select():
    if citation.article_id.title != 'None' and citation.cited_by.title != 'None':
        total_digraph.add_edge(citation.cited_by, citation.article_id)
        
total_digraph.remove_edges_from([edge for edge in total_digraph.selfloop_edges()])

while True:
    try:
        for edge in nx.find_cycle(total_digraph):
            total_digraph.remove_edge(edge[0], edge[1])
    except:
        break
    
        
weakly_subgraphs_list = list(nx.weakly_connected_component_subgraphs(total_digraph))
strongly_subgraphs_list = list(nx.strongly_connected_component_subgraphs(total_digraph))

print('Number of nodes: {0}'.format(len(total_digraph.nodes)),
      'Number of edges: {0}'.format(len(total_digraph.edges)),
      'Number of weakly connected subgraphs: {0}'.format(len(weakly_subgraphs_list)),
      'Number of strongly connected subgraphs: {0}'.format(len(strongly_subgraphs_list)),
       sep='\n')

In [ ]:
root_list = []
keywords = []
citation_factor = 10

for node in total_digraph.nodes:
    if not list(total_digraph.successors(node)):
        if len(list(total_digraph.predecessors(node))) == citation_factor:
            root_list.append(node)
            print(node.title)

for root in root_list:
    parser = ArticleParser()
    keywords.append(parser.parse(root.article_id)['keywords'])

In [ ]:
for ls in keywords:
    for keyword in ls:
        print(keyword, end=', ')
    print()

In [ ]:
def get_the_longest_path(graph, source, target):
    simple_paths = nx.all_simple_paths(graph, source, target)
    the_longest_path = []
    for path in simple_paths:
        if len(path) > len(the_longest_path):
            the_longest_path = path
    return the_longest_path

In [ ]:
the_longest_path_dict = {}

for root in root_list:
    print(root)
    counter = 0
    
    the_longest_path = []
    for node in total_digraph.nodes:
        counter += 1
        print(counter)
        path = get_the_longest_path(total_digraph, node, root)
        if len(path) > len(the_longest_path):
            the_longest_path = path
    the_longest_path_dict[root] = the_longest_path